<a href="https://colab.research.google.com/github/keerthana151204/SACapstoneProject/blob/main/Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pathway bokeh --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import column
from bokeh.models import ColumnDataSource
import pathway as pw
# Configure Bokeh to render plots directly inside the notebook
output_notebook()


In [3]:
# Upload files
from google.colab import files
uploaded = files.upload()

Saving dataset.csv to dataset.csv


In [4]:
df = pd.read_csv("dataset.csv")
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


In [5]:

# Renameing columns to avoid confusion
df.rename(columns={
    "Occupancy": "occupancy",
    "Capacity": "capacity",
    "QueueLength": "queue_length",
    "VehicleType": "vehicle_type",
    "TrafficConditionNearby": "traffic",
    "IsSpecialDay": "is_special_day",
    "Latitude": "latitude",
    "Longitude": "longitude"
}, inplace=True)


In [6]:
traffic_map = {"Low": 0, "Medium": 1, "High": 2} #converting categorical values to numeric
df["traffic"] = df["traffic"].map(traffic_map)


In [7]:
def model1_linear_price(prev_price, occupancy, capacity, alpha=0.5): #Defining a Python function for Model 1: Baseline Linear Pricing
    return prev_price + alpha * (occupancy / capacity) #The new price increases linearly as the parking lot fills, when occupancy = capacity, price increases the most


In [8]:
# Demand-based pricing model with historical and competitive influence
def model2(row, history_occupancy, competitor_price, base_price=10.0, lambd=0.5):
    α, β, γ, δ, ε, ζ, η = 0.4, 0.2, 0.3, 0.1, 0.15, 0.1, 0.1 #Initializing weights
    occ_rate = row["occupancy"] / row["capacity"] #Calculating the occupancy rate as a fraction
    historical_factor = ζ * history_occupancy #Computing a historical occupancy factor by multiplying it with its weight as higher past congestion increases future demand
    competitor_factor = η * (competitor_price - base_price) / base_price  # normalized competitor influence: If a nearby lot is more expensive then positive adjustment and if a nearby lot is cheaper then negative adjustment

    demand = (
        α * occ_rate +                              # Direct demand from current occupancy
        β * row["queue_length"] -                   # Longer queue = higher demand
        γ * row["traffic"] +                        # Traffic discourages demand
        δ * row["is_special_day"] +                 # Special event → increase demand
        ε * vehicle_type_weight(row["vehicle_type"]) + # Truck > Car > Bike
        historical_factor +                         # Demand trend based on past
        competitor_factor                           # Influence from nearby lot prices
    )

    norm_demand = (demand - (-1)) / (4 - (-1))  # Normalizing demand to a 0–1 scale using min-max scaling
    #Assumption: demand can range from -1 to 4 (safe range considering all weights and values)
    price = base_price * (1 + lambd * norm_demand) #Computing the new price based on how strong demand is using formula: price=10*(1+0.5×normalized_demand)
    return np.clip(price, 0.5 * base_price, 2 * base_price) #Setting range from 5 to 20


In [9]:

from math import radians, sin, cos, sqrt, atan2

#function to calculate great-circle distance between two coordinates
def gsdist(lat1, lon1, lat2, lon2):
    R = 6371e3  # Earth radius in meters
    phi1, phi2 = radians(lat1), radians(lat2) #Converting the latitudes to radians so trigonometric functions work
    dphi = radians(lat2 - lat1) #Computing the difference in latitude between two locations
    dlambda = radians(lon2 - lon1) #Computing the difference in longitude between two locations
    a = sin(dphi/2)**2 + cos(phi1) * cos(phi2) * sin(dlambda/2)**2 #Haversine formula: to calculate square of half the chord length between the points (a is a dimensionless value (between 0 and 1) representing angular distance)
    return R * 2 * atan2(sqrt(a), sqrt(1 - a)) #Using atan2 to compute the angular distance in radians and multiplying by R to convert to meters

#fuction to determine if a vehicle should be rerouted
def reroute(lot, all_lots, radius=300):
    suggestions = [] #Initializing an empty list to store recommended rerouting options
    if lot["occupancy"] >= lot["capacity"]: #If not full, skip rerouting
        for _, other in all_lots.iterrows():
            if other["SystemCodeNumber"] != lot["SystemCodeNumber"]:
                dist = gsdist(lot["latitude"], lot["longitude"], other["latitude"], other["longitude"]) #Calculating the distance from the current (full) lot to the other lot using the gsdist() function
                if dist <= radius and other["occupancy"] < other["capacity"] and other["price"] < lot["price"]: #Check three conditions: if the other lot is within 300 meters, if the other lot has available space and if the other lot is cheaper than the current lot. If all are true, consider rerouting
                    suggestions.append((other["SystemCodeNumber"], other["price"], dist))
    return suggestions  # List of (SystemCodeNumber, price, distance)


In [10]:
def vehicle_weight(v_type):
    weights = {"car": 1.0, "bike": 0.5, "truck": 1.5} #Assigning weights based on size
    # Use .get() to return the weight for the given vehicle type (case-insensitive)
    # Default to 1.0 if the type is unknown
    return weights.get(v_type.lower(), 1.0)

# Main pricing model: computes price based on real-time demand signals using linear weighted sum
def model2_demand(row, base_price=10.0, lambd=0.5):
    α, β, γ, δ, ε = 0.4, 0.2, 0.3, 0.1, 0.15 #Assigning weights to the demand components
    occ_rate = row["occupancy"] / row["capacity"]
    #using linear weighted sum model:
    demand = (
        α * occ_rate +
        β * row["queue_length"] -
        γ * row["traffic"] +
        δ * row["is_special_day"] +
        ε * vehicle_weight(row["vehicle_type"])
    )
    norm_demand = (demand - (-1)) / (3 - (-1))  # min-max normalization
    price = base_price * (1 + lambd * norm_demand)
    return np.clip(price, 0.5 * base_price, 2 * base_price)


In [11]:
# Import Bokeh's core plotting tools and output display configuration
from bokeh.plotting import figure, show, output_notebook
# Import ColumnDataSource which links dataframes to Bokeh plots
from bokeh.models import ColumnDataSource
output_notebook()

# Select a single parking lot by filtering all rows where the lot ID matches the first lot in the dataset
lot_df = df[df["SystemCodeNumber"] == df["SystemCodeNumber"].iloc[0]].copy()
# Apply the Model 2 pricing function to every row for the selected lot and save the computed prices
lot_df["price"] = lot_df.apply(lambda row: model2_demand(row), axis=1)

# Create a numeric index for x-axis
lot_df["time_index"] = range(len(lot_df))

# Bokeh plot
source = ColumnDataSource(lot_df)
# Create a line chart figure to visualize price over time
p = figure(height=300, title="Model 2 Price over Time", x_axis_label="Time Index", y_axis_label="Price")
# Draw a blue line connecting all (time_index, price) points
p.line(x='time_index', y='price', source=source, line_width=2)
show(p)
